In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta, date

In [2]:
off = pd.read_csv(r'C:\Users\apath\Documents\MLB Logistic Regression\statcast_metrics.csv')
off.columns = ['Player','Event','Result','Exit Velocity','Pitch Velocity','xBA','Game ID','Date','Team','Inning','Launch Angle','Distance']
off["Date"] = pd.to_datetime(off["Date"])
off = off.sort_values(by='Date')
off.head()


C:\Users\apath\AppData\Local\Temp\ipykernel_25312\4018757100.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  off = pd.read_csv(r'C:\Users\apath\Documents\MLB Logistic Regression\statcast_metrics.csv')


,Player,Event,Result,Exit Velocity,Pitch Velocity,xBA,Game ID,Date,Team,Inning,Launch Angle,Distance
286311,Jose Abreu,36,Strikeout,NaN,82.3,NaN,634655,2021-04-01,White Sox,5,NaN,NaN
146530,Leody Taveras,89,Strikeout,NaN,84.6,NaN,634630,2021-04-01,Rangers,8,NaN,NaN
146531,Eli White,88,Walk,NaN,96.9,NaN,634630,2021-04-01,Rangers,8,NaN,NaN
146532,Brock Holt,87,Walk,NaN,96.1,NaN,634630,2021-04-01,Rangers,8,NaN,NaN
146533,Nathaniel Lowe,86,Strikeout,NaN,95.2,NaN,634630,2021-04-01,Rangers,8,NaN,NaN


In [3]:
res = pd.read_csv(r'C:\Users\apath\Documents\MLB Logistic Regression\general_game.csv')
res.columns = ['Game ID','Date','Team','Opponent','Home Record','Away Record','Home Wins','Away Wins','Home Losses','Away Losses','Home Runs','Away Runs','Home Hits','Away Hits','Home XBA','Away XBA']
res["Date"] = pd.to_datetime(res["Date"])
res = res.sort_values(by='Date')
res = res.replace(['D-backs'],'D-Backs')
res = res.replace(['Red'],'Red Sox')
res = res.replace(['Blue'],'Blue Jays')
res = res.replace(['Indians'],'Guardians')
res = res.replace(['White'],'White Sox')
res


,Game ID,Date,Team,Opponent,Home Record,Away Record,Home Wins,Away Wins,Home Losses,Away Losses,Home Runs,Away Runs,Home Hits,Away Hits,Home XBA,Away XBA
4766,634638,2021-04-01,Brewers,Twins,1-0,0-1,1,0,0,1,6,5,11,10,0.289,0.257
2215,634645,2021-04-01,Tigers,Guardians,1-0,0-1,1,0,0,1,3,2,5,6,0.179,0.224
2306,634622,2021-04-01,Phillies,Braves,1-0,0-1,1,0,0,1,3,2,9,7,0.218,0.270
1659,634640,2021-04-01,Athletics,Astros,0-1,1-0,0,1,1,0,1,8,6,9,0.211,0.344
2363,634618,2021-04-01,Padres,D-Backs,1-0,0-1,1,0,0,1,8,7,10,12,0.266,0.271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,663131,2022-10-05,Reds,Cubs,62-100,74-88,62,74,100,88,2,15,7,13,0.178,0.226
996,661996,2022-10-05,Rangers,Yankees,68-94,99-63,68,99,94,63,4,2,11,5,0.239,0.193
1778,662304,2022-10-05,Pirates,Cardinals,62-100,93-69,62,93,100,69,5,3,9,8,0.267,0.254
3603,661227,2022-10-05,Brewers,D-Backs,86-76,74-88,86,74,76,88,2,4,7,11,0.299,0.239


In [4]:
rslt = res[['Game ID', 'Date', 'Team','Opponent','Home Runs','Away Runs']]
rslt = rslt.rename(columns = {'Home Runs':'Runs'})

rslta = res[['Game ID','Date','Opponent','Team','Away Runs','Home Runs']]
rslta = rslta.rename(columns = {'Opponent':'Team','Team':'Opponent','Away Runs':'Runs','Home Runs':'Away Runs'})

rslt = rslt.append(rslta)

rslt = rslt.sort_values(by='Date')

rslt

C:\Users\apath\AppData\Local\Temp\ipykernel_25312\4248480459.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rslt = rslt.append(rslta)


,Game ID,Date,Team,Opponent,Runs,Away Runs
4766,634638,2021-04-01,Brewers,Twins,6,5
2696,634615,2021-04-01,Dodgers,Rockies,5,8
979,634642,2021-04-01,Blue Jays,Yankees,3,2
1531,634625,2021-04-01,Giants,Mariners,7,8
132,634636,2021-04-01,Cardinals,Reds,11,6
...,...,...,...,...,...,...
1167,663131,2022-10-05,Reds,Cubs,2,15
996,661996,2022-10-05,Rangers,Yankees,4,2
1778,662304,2022-10-05,Pirates,Cardinals,5,3
1458,661427,2022-10-05,White Sox,Twins,1,10


In [5]:
rslt['Outcome']=np.where(rslt['Runs']>rslt['Away Runs'],'Win','Loss')

In [6]:
pit = pd.read_csv(r'C:\Users\apath\Documents\MLB Logistic Regression\pitching_velocity.csv')
pit.columns = ['Pitcher','Batter','Game Pitch','Pitch','Plate Appearance','Inning','Result','Pitch Type','MPH','Spin','Game ID','Date','Team','Team1','Number','VBreak','HBreak']
pit["Date"] = pd.to_datetime(pit["Date"])
pit = pit.sort_values(by='Date')
pit.head()

,Pitcher,Batter,Game Pitch,Pitch,Plate Appearance,Inning,Result,Pitch Type,MPH,Spin,Game ID,Date,Team,Team1,Number,VBreak,HBreak
571530,Wade Davis,Nathaniel Lowe,402,8,101,9,Swinging Strike,Cutter,89.8,2696.0,634630,2021-04-01,Royals,Rangers,1,28,2
453439,Jarlin Garcia,Kyle Seager,259,2,66,8,Ball,4-Seam Fastball,92.5,2231.0,634625,2021-04-01,Mariners,Mariners,87,18,17
453438,Jarlin Garcia,Kyle Seager,260,3,66,8,Called Strike,4-Seam Fastball,91.7,2177.0,634625,2021-04-01,Giants,Giants,86,16,18
453437,Jarlin Garcia,Kyle Seager,261,4,66,8,Called Strike,Slider,84.5,2345.0,634625,2021-04-01,Mariners,Mariners,85,36,3
453436,Jarlin Garcia,Kyle Seager,262,5,66,8,Ball,4-Seam Fastball,92.3,2252.0,634625,2021-04-01,Giants,Giants,84,18,17


In [7]:
#vlookup.... PA, AB, H, TB, BB/HPB/CI

vlookup = [['Lineout',1,1,0,0,0], ['Strikeout',1,1,0,0,0], ['Flyout',1,1,0,0,0], ['Groundout',1,1,0,0,0], ['Pop Out',1,1,0,0,0], ['Single',1,1,1,1,0],
       ['Double',1,1,1,2,0], ['Home Run',1,1,1,4,0], ['Caught Stealing 2B',0,0,0,0,0], ['Walk',1,0,0,0,1],
       ['Catcher Interference',1,0,0,0,1], ['Hit By Pitch',1,0,0,0,1], ['Forceout',1,1,0,0,0], ['Field Error',1,1,0,0,0],
       ['Sac Fly',1,0,0,0,0], ['GIDP',1,1,0,0,0], ['Strikeout Double Play',1,1,0,0,0], ['Fielders Choice',1,1,0,0,0],
       ['Triple',1,1,1,3,0], ['Pickoff 1B',0,0,0,0,0], ['Double Play',1,1,0,0,0], ['Intent Walk',1,0,0,0,1], ['Runner Out',0,0,0,0,0],
       ['Sac Bunt',1,0,0,0,0], ['Sac Fly Double Play',1,1,0,0,0], ['Fielders Choice Out',1,1,0,0,0],
       ['Bunt Pop Out',1,1,0,0,0], ['Pickoff Caught Stealing 2B',0,0,0,0,0],
       ['Caught Stealing Home',0,0,0,0,0], ['Bunt Groundout',1,1,0,0,0], ['Bunt Lineout',1,1,0,0,0],
       ['Triple Play',1,1,0,0,0], ['Caught Stealing 3B',0,0,0,0,0]]

vl = pd.DataFrame(vlookup,columns=['Result','PA','AB','H','TB','BB'])
vl

,Result,PA,AB,H,TB,BB
0,Lineout,1,1,0,0,0
1,Strikeout,1,1,0,0,0
2,Flyout,1,1,0,0,0
3,Groundout,1,1,0,0,0
4,Pop Out,1,1,0,0,0
5,Single,1,1,1,1,0
6,Double,1,1,1,2,0
7,Home Run,1,1,1,4,0
8,Caught Stealing 2B,0,0,0,0,0
9,Walk,1,0,0,0,1


In [8]:
offense = pd.merge(off,vl,how="left", on=["Result"])
offense.head()

,Player,Event,Result,Exit Velocity,Pitch Velocity,xBA,Game ID,Date,Team,Inning,Launch Angle,Distance,PA,AB,H,TB,BB
0,Jose Abreu,36,Strikeout,NaN,82.3,NaN,634655,2021-04-01,White Sox,5,NaN,NaN,1.0,1.0,0.0,0.0,0.0
1,Leody Taveras,89,Strikeout,NaN,84.6,NaN,634630,2021-04-01,Rangers,8,NaN,NaN,1.0,1.0,0.0,0.0,0.0
2,Eli White,88,Walk,NaN,96.9,NaN,634630,2021-04-01,Rangers,8,NaN,NaN,1.0,0.0,0.0,0.0,1.0
3,Brock Holt,87,Walk,NaN,96.1,NaN,634630,2021-04-01,Rangers,8,NaN,NaN,1.0,0.0,0.0,0.0,1.0
4,Nathaniel Lowe,86,Strikeout,NaN,95.2,NaN,634630,2021-04-01,Rangers,8,NaN,NaN,1.0,1.0,0.0,0.0,0.0


In [9]:
pitResults = pit['Result'].unique()
pitResults

array(['Swinging Strike', 'Ball', 'Called Strike', 'Foul', 'Ball In Dirt',
       'In play, run(s)', 'In play, out(s)', 'In play, no out',
       'Hit By Pitch', 'Foul Bunt', 'Missed Bunt', 'Foul Tip', 'Pitchout',
       'Foul Pitchout'], dtype=object)

In [10]:
#vlookup.... Ball, Strike, In play, run(); In play, no out; In Play, out; HBP
vlook = [['Ball',1,0,0,0,0,0], ['In play, no out',0,0,0,1,0,0], ['Swinging Strike',0,1,0,0,0,0], ['Called Strike',0,1,0,0,0,0], ['Foul',0,1,0,0,0,0], ['In play, run(s)',0,0,1,0,0,0],
       ['Ball In Dirt',1,0,0,0,0,1], ['Hit By Pitch',0,0,0,0,0,1], ['Foul Tip',0,1,0,0,0,0], ['Missed Bunt',0,1,0,0,0,0],['In play, out(s)',0,0,0,0,1,0],
       ['Foul Bunt',0,1,0,0,0,0], ['Pitchout',1,0,0,0,0,0],['Foul Pitchout',0,1,0,0,0,0]]
vu = pd.DataFrame(vlook,columns=['Result','Ball','Strike','In play, run(s)','In play, no out','In play, out(s)','Hit By Pitch'])
vu

,Result,Ball,Strike,"In play, run(s)","In play, no out","In play, out(s)",Hit By Pitch
0,Ball,1,0,0,0,0,0
1,"In play, no out",0,0,0,1,0,0
2,Swinging Strike,0,1,0,0,0,0
3,Called Strike,0,1,0,0,0,0
4,Foul,0,1,0,0,0,0
5,"In play, run(s)",0,0,1,0,0,0
6,Ball In Dirt,1,0,0,0,0,1
7,Hit By Pitch,0,0,0,0,0,1
8,Foul Tip,0,1,0,0,0,0
9,Missed Bunt,0,1,0,0,0,0


In [11]:
pitching = pd.merge(pit,vu,how="left", on=["Result"])

pitching.head()

,Pitcher,Batter,Game Pitch,Pitch,Plate Appearance,Inning,Result,Pitch Type,MPH,Spin,...,Team1,Number,VBreak,HBreak,Ball,Strike,"In play, run(s)","In play, no out","In play, out(s)",Hit By Pitch
0,Wade Davis,Nathaniel Lowe,402,8,101,9,Swinging Strike,Cutter,89.8,2696.0,...,Rangers,1,28,2,0,1,0,0,0,0
1,Jarlin Garcia,Kyle Seager,259,2,66,8,Ball,4-Seam Fastball,92.5,2231.0,...,Mariners,87,18,17,1,0,0,0,0,0
2,Jarlin Garcia,Kyle Seager,260,3,66,8,Called Strike,4-Seam Fastball,91.7,2177.0,...,Giants,86,16,18,0,1,0,0,0,0
3,Jarlin Garcia,Kyle Seager,261,4,66,8,Called Strike,Slider,84.5,2345.0,...,Mariners,85,36,3,0,1,0,0,0,0
4,Jarlin Garcia,Kyle Seager,262,5,66,8,Ball,4-Seam Fastball,92.3,2252.0,...,Giants,84,18,17,1,0,0,0,0,0


In [12]:
pitching = pitching.rename(columns={'Team':'Opponent'})
pitching.columns.values.tolist()

['Pitcher',
 'Batter',
 'Game Pitch',
 'Pitch',
 'Plate Appearance',
 'Inning',
 'Result',
 'Pitch Type',
 'MPH',
 'Spin',
 'Game ID',
 'Date',
 'Opponent',
 'Team1',
 'Number',
 'VBreak',
 'HBreak',
 'Ball',
 'Strike',
 'In play, run(s)',
 'In play, no out',
 'In play, out(s)',
 'Hit By Pitch']

In [20]:
day_delta=datetime.timedelta(days=1)
end_date = pd.to_datetime(date(2022,10,1))
start_date = end_date - 30*day_delta
df = pd.DataFrame([])

for i in range((end_date - start_date).days):
    end = start_date + i*day_delta
    start = end - 14*day_delta
    
    
    dfo = offense[offense["Date"].between(start,end)]
    pivotTeamAvg = dfo.pivot_table(index=['Team'],
                        values = ['Exit Velocity','Distance','Launch Angle'],
                        aggfunc = 'mean')
    pivotTeamSum = dfo.pivot_table(index=['Team'],
                        values = ['PA','AB','H','TB','BB'],
                        aggfunc = ['sum'])
    dfta = pivotTeamAvg.reset_index()
    dfts = pivotTeamSum.reset_index()
    dftsa = dfta.join(dfts, on='Team')
    
    
    dfp = pitching[pitching["Date"].between(start,end)] 
    pivotPitAvg = dfp.pivot_table(index=['Opponent'],
                                 values = ['MPH','Spin','VBreak','HBreak'],
                                 aggfunc = 'mean')
    pivotPitSum = dfp.pivot_table(index=['Opponent'],
                                 values = ['Result','Ball','Strike','In play, run(s)','In play, no out','In play, out(s)','Hit By Pitch'],
                                 aggfunc = ['sum'])
    dfpa = pivotPitAvg.reset_index()
    dfps = pivotPitSum.reset_index()
    dfpsa = dfpa.join(dfps, on=["Opponent"])
    
    results = rslt[rslt["Date"].between(end,end)]
    
    dftt = results.join(dftsa,on=["Team"])
    
    dft = dftt.join(dfpsa,on=["Opponent"])
    
    
    df = pd.concat([df,dft])
    

print(df)
df


C:\Users\apath\AppData\Local\Temp\ipykernel_25312\3536812535.py:20: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  dftsa = dfta.join(dfts, on='Team')


ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [14]:
#one hot encoding
#document preprocessing in report and how I extracted features
#ensure scaling is on a similar range.... Normalization will be required
#create different versions of the data for iterations
#Training and validation.  Keep results separate.

#make sure to properly pull in opponent pitching, opposed to team

In [15]:
df = df.replace(np.nan, 0)

In [16]:
dfteams = df['Team'].unique()
dfteams

array(['Blue Jays', 'Cardinals', 'Mets', 'Marlins', 'Angels', 'Brewers',
       'Cubs', 'Pirates', 'Athletics', 'D-Backs', 'Mariners', 'Royals',
       'Red Sox', 'Orioles', 'White Sox', 'Rockies', 'Astros', 'Twins',
       'Reds', 'Dodgers', 'Braves', 'Rays', 'Nationals', 'Giants',
       'Rangers', 'Yankees', 'Tigers', 'Padres', 'Guardians', 'Phillies',
       'NL', 'AL'], dtype=object)

In [17]:
one_hot_encoded_df = pd.get_dummies(df,columns=['Team'])

In [18]:
one_hot_encoded_df.columns.values.tolist()

['Game ID',
 'Date',
 'Opponent',
 'Runs',
 'Away Runs',
 'Outcome',
 'Distance',
 'Exit Velocity',
 'Launch Angle',
 ('sum', 'AB'),
 ('sum', 'BB'),
 ('sum', 'H'),
 ('sum', 'PA'),
 ('sum', 'TB'),
 'HBreak',
 'MPH',
 'Spin',
 'VBreak',
 ('sum', 'Ball'),
 ('sum', 'Hit By Pitch'),
 ('sum', 'In play, no out'),
 ('sum', 'In play, out(s)'),
 ('sum', 'In play, run(s)'),
 ('sum', 'Strike'),
 ('Team', ''),
 ('Opponent', ''),
 'Team_AL',
 'Team_Angels',
 'Team_Astros',
 'Team_Athletics',
 'Team_Blue Jays',
 'Team_Braves',
 'Team_Brewers',
 'Team_Cardinals',
 'Team_Cubs',
 'Team_D-Backs',
 'Team_Dodgers',
 'Team_Giants',
 'Team_Guardians',
 'Team_Mariners',
 'Team_Marlins',
 'Team_Mets',
 'Team_NL',
 'Team_Nationals',
 'Team_Orioles',
 'Team_Padres',
 'Team_Phillies',
 'Team_Pirates',
 'Team_Rangers',
 'Team_Rays',
 'Team_Red Sox',
 'Team_Reds',
 'Team_Rockies',
 'Team_Royals',
 'Team_Tigers',
 'Team_Twins',
 'Team_White Sox',
 'Team_Yankees']

In [283]:
#approach1
X=one_hot_encoded_df[['Game ID','Distance',
 'Exit Velocity',
 'Launch Angle',
 ('sum', 'AB'),
 ('sum', 'BB'),
 ('sum', 'H'),
 ('sum', 'PA'),
 ('sum', 'TB'),'Team_AL',
 'Team_Angels',
 'Team_Astros',
 'Team_Athletics',
 'Team_Blue Jays',
 'Team_Braves',
 'Team_Brewers',
 'Team_Cardinals',
 'Team_Cubs',
 'Team_D-Backs',
 'Team_Dodgers',
 'Team_Giants',
 'Team_Guardians',
 'Team_Mariners',
 'Team_Marlins',
 'Team_Mets',
 'Team_NL',
 'Team_Nationals',
 'Team_Orioles',
 'Team_Padres',
 'Team_Phillies',
 'Team_Pirates',
 'Team_Rangers',
 'Team_Rays',
 'Team_Red Sox',
 'Team_Reds',
 'Team_Rockies',
 'Team_Royals',
 'Team_Tigers',
 'Team_Twins',
 'Team_White Sox',
 'Team_Yankees']]
Y=one_hot_encoded_df['Runs']



C:\Users\apath\anaconda3\lib\site-packages\pandas\core\common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


In [276]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

print(X_train.shape, X_test.shape)

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import sklearn.metrics as sm

model = LinearRegression()

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

mse = mean_squared_error(Y_test, Y_pred)

print(mse)

validation = X_test
validation["Predicted Runs"]=Y_pred

validation['Team']=np.where(validation['Team_AL']==1,'AL',
                            np.where(validation['Team_Angels']==1,'Angels',
                            np.where(validation['Team_Astros']==1,'Astros',
                            np.where(validation['Team_Athletics']==1,'Athletics',
                            np.where(validation['Team_Blue Jays']==1,'Blue Jays',
                            np.where(validation['Team_Braves']==1,'Braves',
                            np.where(validation['Team_Brewers']==1,'Brewers',
                            np.where(validation['Team_Cardinals']==1,'Cardinals',
                            np.where(validation['Team_Cubs']==1,'Cubs',
                            np.where(validation['Team_D-Backs']==1,'D-Backs',
                            np.where(validation['Team_Dodgers']==1,'Dodgers',
                            np.where(validation['Team_Giants']==1,'Giants',         
                            np.where(validation['Team_Guardians']==1,'Guardians',
                            np.where(validation['Team_Mariners']==1,'Mariners',
                            np.where(validation['Team_Marlins']==1,'Marlins',
                            np.where(validation['Team_Mets']==1,'Mets',         
                            np.where(validation['Team_NL']==1,'NL',
                            np.where(validation['Team_Nationals']==1,'Nationals',
                            np.where(validation['Team_Orioles']==1,'Orioles',
                            np.where(validation['Team_Padres']==1,'Padres',         
                            np.where(validation['Team_Phillies']==1,'Phillies',
                            np.where(validation['Team_Pirates']==1,'Pirates',
                            np.where(validation['Team_Rangers']==1,'Rangers',
                            np.where(validation['Team_Rays']==1,'Rays',         
                            np.where(validation['Team_Red Sox']==1,'Red Sox',
                            np.where(validation['Team_Reds']==1,'Reds',
                            np.where(validation['Team_Rockies']==1,'Rockies',
                            np.where(validation['Team_Royals']==1,'Royals',         
                            np.where(validation['Team_Tigers']==1,'Tigers',
                            np.where(validation['Team_Twins']==1,'Twins',
                            np.where(validation['Team_White Sox']==1,'White Sox',
                            np.where(validation['Team_Yankees']==1,'Yankees',         
          'n/a'))))))))))))))))))))))))))))))))


validation = validation[["Game ID", "Team", "Predicted Runs"]]


vdf = pd.merge(validation, rslt, on=['Game ID', 'Team'])

ovalidation = validation[["Game ID", "Team", "Predicted Runs"]]
ovalidation = ovalidation.rename(columns={'Team':'Opponent','Predicted Runs':'Opponent Predicted Runs'})

validationdf = pd.merge(vdf, ovalidation, on=['Game ID', 'Opponent'])
validationdf['Predicted Outcome']=np.where(validationdf['Predicted Runs']>validationdf['Opponent Predicted Runs'],'Win','Loss')
validationdf['Prediction Success']=np.where(validationdf['Outcome']==validationdf['Predicted Outcome'],1,0)

print(validationdf["Prediction Success"].mean())

(7660, 41) (1916, 41)
8.613113276461181
0.578125


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [278]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

print(X_train.shape, X_test.shape)


from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

model = Lasso(alpha=2) # Create a Lasso regression model with alpha=0.5
model.fit(X_train, Y_train) # Train the model on the train set
Y_pred = model.predict(X_test) # Use the model to predict the labels on the test set
print(mean_squared_error(Y_test, Y_pred)) # Compute and print out the MSE on the test set


validation = X_test
validation["Predicted Runs"]=Y_pred

validation['Team']=np.where(validation['Team_AL']==1,'AL',
                            np.where(validation['Team_Angels']==1,'Angels',
                            np.where(validation['Team_Astros']==1,'Astros',
                            np.where(validation['Team_Athletics']==1,'Athletics',
                            np.where(validation['Team_Blue Jays']==1,'Blue Jays',
                            np.where(validation['Team_Braves']==1,'Braves',
                            np.where(validation['Team_Brewers']==1,'Brewers',
                            np.where(validation['Team_Cardinals']==1,'Cardinals',
                            np.where(validation['Team_Cubs']==1,'Cubs',
                            np.where(validation['Team_D-Backs']==1,'D-Backs',
                            np.where(validation['Team_Dodgers']==1,'Dodgers',
                            np.where(validation['Team_Giants']==1,'Giants',         
                            np.where(validation['Team_Guardians']==1,'Guardians',
                            np.where(validation['Team_Mariners']==1,'Mariners',
                            np.where(validation['Team_Marlins']==1,'Marlins',
                            np.where(validation['Team_Mets']==1,'Mets',         
                            np.where(validation['Team_NL']==1,'NL',
                            np.where(validation['Team_Nationals']==1,'Nationals',
                            np.where(validation['Team_Orioles']==1,'Orioles',
                            np.where(validation['Team_Padres']==1,'Padres',         
                            np.where(validation['Team_Phillies']==1,'Phillies',
                            np.where(validation['Team_Pirates']==1,'Pirates',
                            np.where(validation['Team_Rangers']==1,'Rangers',
                            np.where(validation['Team_Rays']==1,'Rays',         
                            np.where(validation['Team_Red Sox']==1,'Red Sox',
                            np.where(validation['Team_Reds']==1,'Reds',
                            np.where(validation['Team_Rockies']==1,'Rockies',
                            np.where(validation['Team_Royals']==1,'Royals',         
                            np.where(validation['Team_Tigers']==1,'Tigers',
                            np.where(validation['Team_Twins']==1,'Twins',
                            np.where(validation['Team_White Sox']==1,'White Sox',
                            np.where(validation['Team_Yankees']==1,'Yankees',         
          'n/a'))))))))))))))))))))))))))))))))


validation = validation[["Game ID", "Team", "Predicted Runs"]]


vdf = pd.merge(validation, rslt, on=['Game ID', 'Team'])

ovalidation = validation[["Game ID", "Team", "Predicted Runs"]]
ovalidation = ovalidation.rename(columns={'Team':'Opponent','Predicted Runs':'Opponent Predicted Runs'})

validationdf = pd.merge(vdf, ovalidation, on=['Game ID', 'Opponent'])
validationdf['Predicted Outcome']=np.where(validationdf['Predicted Runs']>validationdf['Opponent Predicted Runs'],'Win','Loss')
validationdf['Prediction Success']=np.where(validationdf['Outcome']==validationdf['Predicted Outcome'],1,0)

print(validationdf["Prediction Success"].mean())


#compute training error

##Adjust alpha.  Grid search for values that provide the optimal values.  Iterate alpha by 0.05 steps from low to high

#after executing win/loss runs comparison, compare to randomly selected wins/losses and also comparing to team average number of runs

(7660, 41) (1916, 41)
8.680841824681504
0.5885416666666666


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [280]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

print(X_train.shape, X_test.shape)


from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

model = Ridge(alpha=2) # Create a Ridge regression model with alpha=0.5
model.fit(X_train, Y_train) # Train the model on the train set
Y_pred = model.predict(X_test) # Use the model to predict the labels on the test set
print(mean_squared_error(Y_test, Y_pred)) # Compute and print out the MSE on the test set

validation = X_test
validation["Predicted Runs"]=Y_pred

validation['Team']=np.where(validation['Team_AL']==1,'AL',
                            np.where(validation['Team_Angels']==1,'Angels',
                            np.where(validation['Team_Astros']==1,'Astros',
                            np.where(validation['Team_Athletics']==1,'Athletics',
                            np.where(validation['Team_Blue Jays']==1,'Blue Jays',
                            np.where(validation['Team_Braves']==1,'Braves',
                            np.where(validation['Team_Brewers']==1,'Brewers',
                            np.where(validation['Team_Cardinals']==1,'Cardinals',
                            np.where(validation['Team_Cubs']==1,'Cubs',
                            np.where(validation['Team_D-Backs']==1,'D-Backs',
                            np.where(validation['Team_Dodgers']==1,'Dodgers',
                            np.where(validation['Team_Giants']==1,'Giants',         
                            np.where(validation['Team_Guardians']==1,'Guardians',
                            np.where(validation['Team_Mariners']==1,'Mariners',
                            np.where(validation['Team_Marlins']==1,'Marlins',
                            np.where(validation['Team_Mets']==1,'Mets',         
                            np.where(validation['Team_NL']==1,'NL',
                            np.where(validation['Team_Nationals']==1,'Nationals',
                            np.where(validation['Team_Orioles']==1,'Orioles',
                            np.where(validation['Team_Padres']==1,'Padres',         
                            np.where(validation['Team_Phillies']==1,'Phillies',
                            np.where(validation['Team_Pirates']==1,'Pirates',
                            np.where(validation['Team_Rangers']==1,'Rangers',
                            np.where(validation['Team_Rays']==1,'Rays',         
                            np.where(validation['Team_Red Sox']==1,'Red Sox',
                            np.where(validation['Team_Reds']==1,'Reds',
                            np.where(validation['Team_Rockies']==1,'Rockies',
                            np.where(validation['Team_Royals']==1,'Royals',         
                            np.where(validation['Team_Tigers']==1,'Tigers',
                            np.where(validation['Team_Twins']==1,'Twins',
                            np.where(validation['Team_White Sox']==1,'White Sox',
                            np.where(validation['Team_Yankees']==1,'Yankees',         
          'n/a'))))))))))))))))))))))))))))))))


validation = validation[["Game ID", "Team", "Predicted Runs"]]


vdf = pd.merge(validation, rslt, on=['Game ID', 'Team'])

ovalidation = validation[["Game ID", "Team", "Predicted Runs"]]
ovalidation = ovalidation.rename(columns={'Team':'Opponent','Predicted Runs':'Opponent Predicted Runs'})

validationdf = pd.merge(vdf, ovalidation, on=['Game ID', 'Opponent'])
validationdf['Predicted Outcome']=np.where(validationdf['Predicted Runs']>validationdf['Opponent Predicted Runs'],'Win','Loss')
validationdf['Prediction Success']=np.where(validationdf['Outcome']==validationdf['Predicted Outcome'],1,0)

print(validationdf["Prediction Success"].mean())

(7660, 41) (1916, 41)
8.61068568551172
0.5833333333333334


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [284]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

print(X_train.shape, X_test.shape)

from sklearn import neighbors

# Set the number of neighbors and the inverse weight function
n_neighbors = 5
knn = neighbors.KNeighborsRegressor(n_neighbors, weights = 'distance')
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
print(mean_squared_error(Y_test, Y_pred)) # Compute and print out the MSE on the test set

validation = X_test
validation["Predicted Runs"]=Y_pred

validation['Team']=np.where(validation['Team_AL']==1,'AL',
                            np.where(validation['Team_Angels']==1,'Angels',
                            np.where(validation['Team_Astros']==1,'Astros',
                            np.where(validation['Team_Athletics']==1,'Athletics',
                            np.where(validation['Team_Blue Jays']==1,'Blue Jays',
                            np.where(validation['Team_Braves']==1,'Braves',
                            np.where(validation['Team_Brewers']==1,'Brewers',
                            np.where(validation['Team_Cardinals']==1,'Cardinals',
                            np.where(validation['Team_Cubs']==1,'Cubs',
                            np.where(validation['Team_D-Backs']==1,'D-Backs',
                            np.where(validation['Team_Dodgers']==1,'Dodgers',
                            np.where(validation['Team_Giants']==1,'Giants',         
                            np.where(validation['Team_Guardians']==1,'Guardians',
                            np.where(validation['Team_Mariners']==1,'Mariners',
                            np.where(validation['Team_Marlins']==1,'Marlins',
                            np.where(validation['Team_Mets']==1,'Mets',         
                            np.where(validation['Team_NL']==1,'NL',
                            np.where(validation['Team_Nationals']==1,'Nationals',
                            np.where(validation['Team_Orioles']==1,'Orioles',
                            np.where(validation['Team_Padres']==1,'Padres',         
                            np.where(validation['Team_Phillies']==1,'Phillies',
                            np.where(validation['Team_Pirates']==1,'Pirates',
                            np.where(validation['Team_Rangers']==1,'Rangers',
                            np.where(validation['Team_Rays']==1,'Rays',         
                            np.where(validation['Team_Red Sox']==1,'Red Sox',
                            np.where(validation['Team_Reds']==1,'Reds',
                            np.where(validation['Team_Rockies']==1,'Rockies',
                            np.where(validation['Team_Royals']==1,'Royals',         
                            np.where(validation['Team_Tigers']==1,'Tigers',
                            np.where(validation['Team_Twins']==1,'Twins',
                            np.where(validation['Team_White Sox']==1,'White Sox',
                            np.where(validation['Team_Yankees']==1,'Yankees',         
          'n/a'))))))))))))))))))))))))))))))))


validation = validation[["Game ID", "Team", "Predicted Runs"]]


vdf = pd.merge(validation, rslt, on=['Game ID', 'Team'])

ovalidation = validation[["Game ID", "Team", "Predicted Runs"]]
ovalidation = ovalidation.rename(columns={'Team':'Opponent','Predicted Runs':'Opponent Predicted Runs'})

validationdf = pd.merge(vdf, ovalidation, on=['Game ID', 'Opponent'])
validationdf['Predicted Outcome']=np.where(validationdf['Predicted Runs']>validationdf['Opponent Predicted Runs'],'Win','Loss')
validationdf['Prediction Success']=np.where(validationdf['Outcome']==validationdf['Predicted Outcome'],1,0)

print(validationdf["Prediction Success"].mean())

(7660, 41) (1916, 41)


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


11.198494399576472
0.4791666666666667


In [285]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

print(X_train.shape, X_test.shape)

from sklearn import tree

model = tree.DecisionTreeRegressor()
model = model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

print(mean_squared_error(Y_test, Y_pred)) # Compute and print out the MSE on the test set

validation = X_test
validation["Predicted Runs"]=Y_pred

validation['Team']=np.where(validation['Team_AL']==1,'AL',
                            np.where(validation['Team_Angels']==1,'Angels',
                            np.where(validation['Team_Astros']==1,'Astros',
                            np.where(validation['Team_Athletics']==1,'Athletics',
                            np.where(validation['Team_Blue Jays']==1,'Blue Jays',
                            np.where(validation['Team_Braves']==1,'Braves',
                            np.where(validation['Team_Brewers']==1,'Brewers',
                            np.where(validation['Team_Cardinals']==1,'Cardinals',
                            np.where(validation['Team_Cubs']==1,'Cubs',
                            np.where(validation['Team_D-Backs']==1,'D-Backs',
                            np.where(validation['Team_Dodgers']==1,'Dodgers',
                            np.where(validation['Team_Giants']==1,'Giants',         
                            np.where(validation['Team_Guardians']==1,'Guardians',
                            np.where(validation['Team_Mariners']==1,'Mariners',
                            np.where(validation['Team_Marlins']==1,'Marlins',
                            np.where(validation['Team_Mets']==1,'Mets',         
                            np.where(validation['Team_NL']==1,'NL',
                            np.where(validation['Team_Nationals']==1,'Nationals',
                            np.where(validation['Team_Orioles']==1,'Orioles',
                            np.where(validation['Team_Padres']==1,'Padres',         
                            np.where(validation['Team_Phillies']==1,'Phillies',
                            np.where(validation['Team_Pirates']==1,'Pirates',
                            np.where(validation['Team_Rangers']==1,'Rangers',
                            np.where(validation['Team_Rays']==1,'Rays',         
                            np.where(validation['Team_Red Sox']==1,'Red Sox',
                            np.where(validation['Team_Reds']==1,'Reds',
                            np.where(validation['Team_Rockies']==1,'Rockies',
                            np.where(validation['Team_Royals']==1,'Royals',         
                            np.where(validation['Team_Tigers']==1,'Tigers',
                            np.where(validation['Team_Twins']==1,'Twins',
                            np.where(validation['Team_White Sox']==1,'White Sox',
                            np.where(validation['Team_Yankees']==1,'Yankees',         
          'n/a'))))))))))))))))))))))))))))))))


validation = validation[["Game ID", "Team", "Predicted Runs"]]


vdf = pd.merge(validation, rslt, on=['Game ID', 'Team'])

ovalidation = validation[["Game ID", "Team", "Predicted Runs"]]
ovalidation = ovalidation.rename(columns={'Team':'Opponent','Predicted Runs':'Opponent Predicted Runs'})

validationdf = pd.merge(vdf, ovalidation, on=['Game ID', 'Opponent'])
validationdf['Predicted Outcome']=np.where(validationdf['Predicted Runs']>validationdf['Opponent Predicted Runs'],'Win','Loss')
validationdf['Prediction Success']=np.where(validationdf['Outcome']==validationdf['Predicted Outcome'],1,0)

print(validationdf["Prediction Success"].mean())

(7660, 41) (1916, 41)
19.05741127348643
0.4947916666666667


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [286]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

print(X_train.shape, X_test.shape)

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

svr = SVR().fit(X_train, Y_train)
Y_pred = svr.predict(X_test)

print(mean_squared_error(Y_test, Y_pred)) # Compute and print out the MSE on the test set

validation = X_test
validation["Predicted Runs"]=Y_pred

validation['Team']=np.where(validation['Team_AL']==1,'AL',
                            np.where(validation['Team_Angels']==1,'Angels',
                            np.where(validation['Team_Astros']==1,'Astros',
                            np.where(validation['Team_Athletics']==1,'Athletics',
                            np.where(validation['Team_Blue Jays']==1,'Blue Jays',
                            np.where(validation['Team_Braves']==1,'Braves',
                            np.where(validation['Team_Brewers']==1,'Brewers',
                            np.where(validation['Team_Cardinals']==1,'Cardinals',
                            np.where(validation['Team_Cubs']==1,'Cubs',
                            np.where(validation['Team_D-Backs']==1,'D-Backs',
                            np.where(validation['Team_Dodgers']==1,'Dodgers',
                            np.where(validation['Team_Giants']==1,'Giants',         
                            np.where(validation['Team_Guardians']==1,'Guardians',
                            np.where(validation['Team_Mariners']==1,'Mariners',
                            np.where(validation['Team_Marlins']==1,'Marlins',
                            np.where(validation['Team_Mets']==1,'Mets',         
                            np.where(validation['Team_NL']==1,'NL',
                            np.where(validation['Team_Nationals']==1,'Nationals',
                            np.where(validation['Team_Orioles']==1,'Orioles',
                            np.where(validation['Team_Padres']==1,'Padres',         
                            np.where(validation['Team_Phillies']==1,'Phillies',
                            np.where(validation['Team_Pirates']==1,'Pirates',
                            np.where(validation['Team_Rangers']==1,'Rangers',
                            np.where(validation['Team_Rays']==1,'Rays',         
                            np.where(validation['Team_Red Sox']==1,'Red Sox',
                            np.where(validation['Team_Reds']==1,'Reds',
                            np.where(validation['Team_Rockies']==1,'Rockies',
                            np.where(validation['Team_Royals']==1,'Royals',         
                            np.where(validation['Team_Tigers']==1,'Tigers',
                            np.where(validation['Team_Twins']==1,'Twins',
                            np.where(validation['Team_White Sox']==1,'White Sox',
                            np.where(validation['Team_Yankees']==1,'Yankees',         
          'n/a'))))))))))))))))))))))))))))))))


validation = validation[["Game ID", "Team", "Predicted Runs"]]


vdf = pd.merge(validation, rslt, on=['Game ID', 'Team'])

ovalidation = validation[["Game ID", "Team", "Predicted Runs"]]
ovalidation = ovalidation.rename(columns={'Team':'Opponent','Predicted Runs':'Opponent Predicted Runs'})

validationdf = pd.merge(vdf, ovalidation, on=['Game ID', 'Opponent'])
validationdf['Predicted Outcome']=np.where(validationdf['Predicted Runs']>validationdf['Opponent Predicted Runs'],'Win','Loss')
validationdf['Prediction Success']=np.where(validationdf['Outcome']==validationdf['Predicted Outcome'],1,0)

print(validationdf["Prediction Success"].mean())

(7660, 41) (1916, 41)


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


9.47002054482546
0.5572916666666666


In [144]:
## Look for regression trees and attempt this model
## SVM regression


In [289]:
#approach2
X=one_hot_encoded_df[['Game ID','Distance',
 'Exit Velocity',
 'Launch Angle',
 ('sum', 'AB'),
 ('sum', 'BB'),
 ('sum', 'H'),
 ('sum', 'PA'),
 ('sum', 'TB'),
 'HBreak',
 'MPH',
 'Spin',
 'VBreak',
 ('sum', 'Ball'),
 ('sum', 'Hit By Pitch'),
 ('sum', 'In play, no out'),
 ('sum', 'In play, out(s)'),
 ('sum', 'In play, run(s)'),
 ('sum', 'Strike'),'Team_AL',
 'Team_Angels',
 'Team_Astros',
 'Team_Athletics',
 'Team_Blue Jays',
 'Team_Braves',
 'Team_Brewers',
 'Team_Cardinals',
 'Team_Cubs',
 'Team_D-Backs',
 'Team_Dodgers',
 'Team_Giants',
 'Team_Guardians',
 'Team_Mariners',
 'Team_Marlins',
 'Team_Mets',
 'Team_NL',
 'Team_Nationals',
 'Team_Orioles',
 'Team_Padres',
 'Team_Phillies',
 'Team_Pirates',
 'Team_Rangers',
 'Team_Rays',
 'Team_Red Sox',
 'Team_Reds',
 'Team_Rockies',
 'Team_Royals',
 'Team_Tigers',
 'Team_Twins',
 'Team_White Sox',
 'Team_Yankees']]
Y=one_hot_encoded_df['Runs']


C:\Users\apath\anaconda3\lib\site-packages\pandas\core\common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


In [290]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=1)

print(X_train.shape, X_test.shape)

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import sklearn.metrics as sm

model = LinearRegression()

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

mse = mean_squared_error(Y_test, Y_pred)

print(mse)


validation = X_test
validation["Predicted Runs"]=Y_pred

validation['Team']=np.where(validation['Team_AL']==1,'AL',
                            np.where(validation['Team_Angels']==1,'Angels',
                            np.where(validation['Team_Astros']==1,'Astros',
                            np.where(validation['Team_Athletics']==1,'Athletics',
                            np.where(validation['Team_Blue Jays']==1,'Blue Jays',
                            np.where(validation['Team_Braves']==1,'Braves',
                            np.where(validation['Team_Brewers']==1,'Brewers',
                            np.where(validation['Team_Cardinals']==1,'Cardinals',
                            np.where(validation['Team_Cubs']==1,'Cubs',
                            np.where(validation['Team_D-Backs']==1,'D-Backs',
                            np.where(validation['Team_Dodgers']==1,'Dodgers',
                            np.where(validation['Team_Giants']==1,'Giants',         
                            np.where(validation['Team_Guardians']==1,'Guardians',
                            np.where(validation['Team_Mariners']==1,'Mariners',
                            np.where(validation['Team_Marlins']==1,'Marlins',
                            np.where(validation['Team_Mets']==1,'Mets',         
                            np.where(validation['Team_NL']==1,'NL',
                            np.where(validation['Team_Nationals']==1,'Nationals',
                            np.where(validation['Team_Orioles']==1,'Orioles',
                            np.where(validation['Team_Padres']==1,'Padres',         
                            np.where(validation['Team_Phillies']==1,'Phillies',
                            np.where(validation['Team_Pirates']==1,'Pirates',
                            np.where(validation['Team_Rangers']==1,'Rangers',
                            np.where(validation['Team_Rays']==1,'Rays',         
                            np.where(validation['Team_Red Sox']==1,'Red Sox',
                            np.where(validation['Team_Reds']==1,'Reds',
                            np.where(validation['Team_Rockies']==1,'Rockies',
                            np.where(validation['Team_Royals']==1,'Royals',         
                            np.where(validation['Team_Tigers']==1,'Tigers',
                            np.where(validation['Team_Twins']==1,'Twins',
                            np.where(validation['Team_White Sox']==1,'White Sox',
                            np.where(validation['Team_Yankees']==1,'Yankees',         
          'n/a'))))))))))))))))))))))))))))))))


validation = validation[["Game ID", "Team", "Predicted Runs"]]


vdf = pd.merge(validation, rslt, on=['Game ID', 'Team'])

ovalidation = validation[["Game ID", "Team", "Predicted Runs"]]
ovalidation = ovalidation.rename(columns={'Team':'Opponent','Predicted Runs':'Opponent Predicted Runs'})

validationdf = pd.merge(vdf, ovalidation, on=['Game ID', 'Opponent'])
validationdf['Predicted Outcome']=np.where(validationdf['Predicted Runs']>validationdf['Opponent Predicted Runs'],'Win','Loss')
validationdf['Prediction Success']=np.where(validationdf['Outcome']==validationdf['Predicted Outcome'],1,0)

print(validationdf["Prediction Success"].mean())

(7660, 51) (1916, 51)
8.391828514148273
0.6145833333333334


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [230]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=1)

print(X_train.shape, X_test.shape)

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

model = Lasso(alpha=1.8) # Create a Lasso regression model with alpha=0.5
model.fit(X_train, Y_train) # Train the model on the train set
Y_pred = model.predict(X_test) # Use the model to predict the labels on the test set
print(mean_squared_error(Y_test, Y_pred)) # Compute and print out the MSE on the test set

validation = X_test
validation["Predicted Runs"]=Y_pred

validation['Team']=np.where(validation['Team_AL']==1,'AL',
                            np.where(validation['Team_Angels']==1,'Angels',
                            np.where(validation['Team_Astros']==1,'Astros',
                            np.where(validation['Team_Athletics']==1,'Athletics',
                            np.where(validation['Team_Blue Jays']==1,'Blue Jays',
                            np.where(validation['Team_Braves']==1,'Braves',
                            np.where(validation['Team_Brewers']==1,'Brewers',
                            np.where(validation['Team_Cardinals']==1,'Cardinals',
                            np.where(validation['Team_Cubs']==1,'Cubs',
                            np.where(validation['Team_D-Backs']==1,'D-Backs',
                            np.where(validation['Team_Dodgers']==1,'Dodgers',
                            np.where(validation['Team_Giants']==1,'Giants',         
                            np.where(validation['Team_Guardians']==1,'Guardians',
                            np.where(validation['Team_Mariners']==1,'Mariners',
                            np.where(validation['Team_Marlins']==1,'Marlins',
                            np.where(validation['Team_Mets']==1,'Mets',         
                            np.where(validation['Team_NL']==1,'NL',
                            np.where(validation['Team_Nationals']==1,'Nationals',
                            np.where(validation['Team_Orioles']==1,'Orioles',
                            np.where(validation['Team_Padres']==1,'Padres',         
                            np.where(validation['Team_Phillies']==1,'Phillies',
                            np.where(validation['Team_Pirates']==1,'Pirates',
                            np.where(validation['Team_Rangers']==1,'Rangers',
                            np.where(validation['Team_Rays']==1,'Rays',         
                            np.where(validation['Team_Red Sox']==1,'Red Sox',
                            np.where(validation['Team_Reds']==1,'Reds',
                            np.where(validation['Team_Rockies']==1,'Rockies',
                            np.where(validation['Team_Royals']==1,'Royals',         
                            np.where(validation['Team_Tigers']==1,'Tigers',
                            np.where(validation['Team_Twins']==1,'Twins',
                            np.where(validation['Team_White Sox']==1,'White Sox',
                            np.where(validation['Team_Yankees']==1,'Yankees',         
          'n/a'))))))))))))))))))))))))))))))))


validation = validation[["Game ID", "Team", "Predicted Runs"]]


vdf = pd.merge(validation, rslt, on=['Game ID', 'Team'])

ovalidation = validation[["Game ID", "Team", "Predicted Runs"]]
ovalidation = ovalidation.rename(columns={'Team':'Opponent','Predicted Runs':'Opponent Predicted Runs'})

validationdf = pd.merge(vdf, ovalidation, on=['Game ID', 'Opponent'])
validationdf['Predicted Outcome']=np.where(validationdf['Predicted Runs']>validationdf['Opponent Predicted Runs'],'Win','Loss')
validationdf['Prediction Success']=np.where(validationdf['Outcome']==validationdf['Predicted Outcome'],1,0)

print(validationdf["Prediction Success"].mean())

(7660, 51) (1916, 51)
8.504545188815134
0.6041666666666666


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [274]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=1)

print(X_train.shape, X_test.shape)

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

model = Ridge(alpha=4.3) # Create a Ridge regression model with alpha=0.5
model.fit(X_train, Y_train) # Train the model on the train set
Y_pred = model.predict(X_test) # Use the model to predict the labels on the test set
print(mean_squared_error(Y_test, Y_pred)) # Compute and print out the MSE on the test set

validation = X_test
validation["Predicted Runs"]=Y_pred

validation['Team']=np.where(validation['Team_AL']==1,'AL',
                            np.where(validation['Team_Angels']==1,'Angels',
                            np.where(validation['Team_Astros']==1,'Astros',
                            np.where(validation['Team_Athletics']==1,'Athletics',
                            np.where(validation['Team_Blue Jays']==1,'Blue Jays',
                            np.where(validation['Team_Braves']==1,'Braves',
                            np.where(validation['Team_Brewers']==1,'Brewers',
                            np.where(validation['Team_Cardinals']==1,'Cardinals',
                            np.where(validation['Team_Cubs']==1,'Cubs',
                            np.where(validation['Team_D-Backs']==1,'D-Backs',
                            np.where(validation['Team_Dodgers']==1,'Dodgers',
                            np.where(validation['Team_Giants']==1,'Giants',         
                            np.where(validation['Team_Guardians']==1,'Guardians',
                            np.where(validation['Team_Mariners']==1,'Mariners',
                            np.where(validation['Team_Marlins']==1,'Marlins',
                            np.where(validation['Team_Mets']==1,'Mets',         
                            np.where(validation['Team_NL']==1,'NL',
                            np.where(validation['Team_Nationals']==1,'Nationals',
                            np.where(validation['Team_Orioles']==1,'Orioles',
                            np.where(validation['Team_Padres']==1,'Padres',         
                            np.where(validation['Team_Phillies']==1,'Phillies',
                            np.where(validation['Team_Pirates']==1,'Pirates',
                            np.where(validation['Team_Rangers']==1,'Rangers',
                            np.where(validation['Team_Rays']==1,'Rays',         
                            np.where(validation['Team_Red Sox']==1,'Red Sox',
                            np.where(validation['Team_Reds']==1,'Reds',
                            np.where(validation['Team_Rockies']==1,'Rockies',
                            np.where(validation['Team_Royals']==1,'Royals',         
                            np.where(validation['Team_Tigers']==1,'Tigers',
                            np.where(validation['Team_Twins']==1,'Twins',
                            np.where(validation['Team_White Sox']==1,'White Sox',
                            np.where(validation['Team_Yankees']==1,'Yankees',         
          'n/a'))))))))))))))))))))))))))))))))


validation = validation[["Game ID", "Team", "Predicted Runs"]]


vdf = pd.merge(validation, rslt, on=['Game ID', 'Team'])

ovalidation = validation[["Game ID", "Team", "Predicted Runs"]]
ovalidation = ovalidation.rename(columns={'Team':'Opponent','Predicted Runs':'Opponent Predicted Runs'})

validationdf = pd.merge(vdf, ovalidation, on=['Game ID', 'Opponent'])
validationdf['Predicted Outcome']=np.where(validationdf['Predicted Runs']>validationdf['Opponent Predicted Runs'],'Win','Loss')
validationdf['Prediction Success']=np.where(validationdf['Outcome']==validationdf['Predicted Outcome'],1,0)

print(validationdf["Prediction Success"].mean())

(7660, 51) (1916, 51)
8.39534202445981
0.6145833333333334


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [292]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

print(X_train.shape, X_test.shape)

from sklearn import neighbors

# Set the number of neighbors and the inverse weight function
n_neighbors = 5
knn = neighbors.KNeighborsRegressor(n_neighbors, weights = 'distance')
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
print(mean_squared_error(Y_test, Y_pred)) # Compute and print out the MSE on the test set

validation = X_test
validation["Predicted Runs"]=Y_pred

validation['Team']=np.where(validation['Team_AL']==1,'AL',
                            np.where(validation['Team_Angels']==1,'Angels',
                            np.where(validation['Team_Astros']==1,'Astros',
                            np.where(validation['Team_Athletics']==1,'Athletics',
                            np.where(validation['Team_Blue Jays']==1,'Blue Jays',
                            np.where(validation['Team_Braves']==1,'Braves',
                            np.where(validation['Team_Brewers']==1,'Brewers',
                            np.where(validation['Team_Cardinals']==1,'Cardinals',
                            np.where(validation['Team_Cubs']==1,'Cubs',
                            np.where(validation['Team_D-Backs']==1,'D-Backs',
                            np.where(validation['Team_Dodgers']==1,'Dodgers',
                            np.where(validation['Team_Giants']==1,'Giants',         
                            np.where(validation['Team_Guardians']==1,'Guardians',
                            np.where(validation['Team_Mariners']==1,'Mariners',
                            np.where(validation['Team_Marlins']==1,'Marlins',
                            np.where(validation['Team_Mets']==1,'Mets',         
                            np.where(validation['Team_NL']==1,'NL',
                            np.where(validation['Team_Nationals']==1,'Nationals',
                            np.where(validation['Team_Orioles']==1,'Orioles',
                            np.where(validation['Team_Padres']==1,'Padres',         
                            np.where(validation['Team_Phillies']==1,'Phillies',
                            np.where(validation['Team_Pirates']==1,'Pirates',
                            np.where(validation['Team_Rangers']==1,'Rangers',
                            np.where(validation['Team_Rays']==1,'Rays',         
                            np.where(validation['Team_Red Sox']==1,'Red Sox',
                            np.where(validation['Team_Reds']==1,'Reds',
                            np.where(validation['Team_Rockies']==1,'Rockies',
                            np.where(validation['Team_Royals']==1,'Royals',         
                            np.where(validation['Team_Tigers']==1,'Tigers',
                            np.where(validation['Team_Twins']==1,'Twins',
                            np.where(validation['Team_White Sox']==1,'White Sox',
                            np.where(validation['Team_Yankees']==1,'Yankees',         
          'n/a'))))))))))))))))))))))))))))))))


validation = validation[["Game ID", "Team", "Predicted Runs"]]


vdf = pd.merge(validation, rslt, on=['Game ID', 'Team'])

ovalidation = validation[["Game ID", "Team", "Predicted Runs"]]
ovalidation = ovalidation.rename(columns={'Team':'Opponent','Predicted Runs':'Opponent Predicted Runs'})

validationdf = pd.merge(vdf, ovalidation, on=['Game ID', 'Opponent'])
validationdf['Predicted Outcome']=np.where(validationdf['Predicted Runs']>validationdf['Opponent Predicted Runs'],'Win','Loss')
validationdf['Prediction Success']=np.where(validationdf['Outcome']==validationdf['Predicted Outcome'],1,0)

print(validationdf["Prediction Success"].mean())

(7660, 51) (1916, 51)


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


11.291588928709222
0.5260416666666666


In [293]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

print(X_train.shape, X_test.shape)

from sklearn import tree

model = tree.DecisionTreeRegressor()
model = model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

print(mean_squared_error(Y_test, Y_pred)) # Compute and print out the MSE on the test set

validation = X_test
validation["Predicted Runs"]=Y_pred

validation['Team']=np.where(validation['Team_AL']==1,'AL',
                            np.where(validation['Team_Angels']==1,'Angels',
                            np.where(validation['Team_Astros']==1,'Astros',
                            np.where(validation['Team_Athletics']==1,'Athletics',
                            np.where(validation['Team_Blue Jays']==1,'Blue Jays',
                            np.where(validation['Team_Braves']==1,'Braves',
                            np.where(validation['Team_Brewers']==1,'Brewers',
                            np.where(validation['Team_Cardinals']==1,'Cardinals',
                            np.where(validation['Team_Cubs']==1,'Cubs',
                            np.where(validation['Team_D-Backs']==1,'D-Backs',
                            np.where(validation['Team_Dodgers']==1,'Dodgers',
                            np.where(validation['Team_Giants']==1,'Giants',         
                            np.where(validation['Team_Guardians']==1,'Guardians',
                            np.where(validation['Team_Mariners']==1,'Mariners',
                            np.where(validation['Team_Marlins']==1,'Marlins',
                            np.where(validation['Team_Mets']==1,'Mets',         
                            np.where(validation['Team_NL']==1,'NL',
                            np.where(validation['Team_Nationals']==1,'Nationals',
                            np.where(validation['Team_Orioles']==1,'Orioles',
                            np.where(validation['Team_Padres']==1,'Padres',         
                            np.where(validation['Team_Phillies']==1,'Phillies',
                            np.where(validation['Team_Pirates']==1,'Pirates',
                            np.where(validation['Team_Rangers']==1,'Rangers',
                            np.where(validation['Team_Rays']==1,'Rays',         
                            np.where(validation['Team_Red Sox']==1,'Red Sox',
                            np.where(validation['Team_Reds']==1,'Reds',
                            np.where(validation['Team_Rockies']==1,'Rockies',
                            np.where(validation['Team_Royals']==1,'Royals',         
                            np.where(validation['Team_Tigers']==1,'Tigers',
                            np.where(validation['Team_Twins']==1,'Twins',
                            np.where(validation['Team_White Sox']==1,'White Sox',
                            np.where(validation['Team_Yankees']==1,'Yankees',         
          'n/a'))))))))))))))))))))))))))))))))


validation = validation[["Game ID", "Team", "Predicted Runs"]]


vdf = pd.merge(validation, rslt, on=['Game ID', 'Team'])

ovalidation = validation[["Game ID", "Team", "Predicted Runs"]]
ovalidation = ovalidation.rename(columns={'Team':'Opponent','Predicted Runs':'Opponent Predicted Runs'})

validationdf = pd.merge(vdf, ovalidation, on=['Game ID', 'Opponent'])
validationdf['Predicted Outcome']=np.where(validationdf['Predicted Runs']>validationdf['Opponent Predicted Runs'],'Win','Loss')
validationdf['Prediction Success']=np.where(validationdf['Outcome']==validationdf['Predicted Outcome'],1,0)

print(validationdf["Prediction Success"].mean())

(7660, 51) (1916, 51)
18.605949895615865
0.4947916666666667


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [291]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

print(X_train.shape, X_test.shape)

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

svr = SVR().fit(X_train, Y_train)
Y_pred = svr.predict(X_test)

print(mean_squared_error(Y_test, Y_pred)) # Compute and print out the MSE on the test set

validation = X_test
validation["Predicted Runs"]=Y_pred

validation['Team']=np.where(validation['Team_AL']==1,'AL',
                            np.where(validation['Team_Angels']==1,'Angels',
                            np.where(validation['Team_Astros']==1,'Astros',
                            np.where(validation['Team_Athletics']==1,'Athletics',
                            np.where(validation['Team_Blue Jays']==1,'Blue Jays',
                            np.where(validation['Team_Braves']==1,'Braves',
                            np.where(validation['Team_Brewers']==1,'Brewers',
                            np.where(validation['Team_Cardinals']==1,'Cardinals',
                            np.where(validation['Team_Cubs']==1,'Cubs',
                            np.where(validation['Team_D-Backs']==1,'D-Backs',
                            np.where(validation['Team_Dodgers']==1,'Dodgers',
                            np.where(validation['Team_Giants']==1,'Giants',         
                            np.where(validation['Team_Guardians']==1,'Guardians',
                            np.where(validation['Team_Mariners']==1,'Mariners',
                            np.where(validation['Team_Marlins']==1,'Marlins',
                            np.where(validation['Team_Mets']==1,'Mets',         
                            np.where(validation['Team_NL']==1,'NL',
                            np.where(validation['Team_Nationals']==1,'Nationals',
                            np.where(validation['Team_Orioles']==1,'Orioles',
                            np.where(validation['Team_Padres']==1,'Padres',         
                            np.where(validation['Team_Phillies']==1,'Phillies',
                            np.where(validation['Team_Pirates']==1,'Pirates',
                            np.where(validation['Team_Rangers']==1,'Rangers',
                            np.where(validation['Team_Rays']==1,'Rays',         
                            np.where(validation['Team_Red Sox']==1,'Red Sox',
                            np.where(validation['Team_Reds']==1,'Reds',
                            np.where(validation['Team_Rockies']==1,'Rockies',
                            np.where(validation['Team_Royals']==1,'Royals',         
                            np.where(validation['Team_Tigers']==1,'Tigers',
                            np.where(validation['Team_Twins']==1,'Twins',
                            np.where(validation['Team_White Sox']==1,'White Sox',
                            np.where(validation['Team_Yankees']==1,'Yankees',         
          'n/a'))))))))))))))))))))))))))))))))


validation = validation[["Game ID", "Team", "Predicted Runs"]]


vdf = pd.merge(validation, rslt, on=['Game ID', 'Team'])

ovalidation = validation[["Game ID", "Team", "Predicted Runs"]]
ovalidation = ovalidation.rename(columns={'Team':'Opponent','Predicted Runs':'Opponent Predicted Runs'})

validationdf = pd.merge(vdf, ovalidation, on=['Game ID', 'Opponent'])
validationdf['Predicted Outcome']=np.where(validationdf['Predicted Runs']>validationdf['Opponent Predicted Runs'],'Win','Loss')
validationdf['Prediction Success']=np.where(validationdf['Outcome']==validationdf['Predicted Outcome'],1,0)

print(validationdf["Prediction Success"].mean())

(7660, 51) (1916, 51)


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


9.46990148823833
0.5833333333333334


In [53]:
#Logistic Regression Attempt
#approach1
X=one_hot_encoded_df[['Game ID','Distance',
 'Exit Velocity',
 'Launch Angle',
 ('sum', 'AB'),
 ('sum', 'BB'),
 ('sum', 'H'),
 ('sum', 'PA'),
 ('sum', 'TB'),'Team_AL',
 'Team_Angels',
 'Team_Astros',
 'Team_Athletics',
 'Team_Blue Jays',
 'Team_Braves',
 'Team_Brewers',
 'Team_Cardinals',
 'Team_Cubs',
 'Team_D-Backs',
 'Team_Dodgers',
 'Team_Giants',
 'Team_Guardians',
 'Team_Mariners',
 'Team_Marlins',
 'Team_Mets',
 'Team_NL',
 'Team_Nationals',
 'Team_Orioles',
 'Team_Padres',
 'Team_Phillies',
 'Team_Pirates',
 'Team_Rangers',
 'Team_Rays',
 'Team_Red Sox',
 'Team_Reds',
 'Team_Rockies',
 'Team_Royals',
 'Team_Tigers',
 'Team_Twins',
 'Team_White Sox',
 'Team_Yankees']]
Y=one_hot_encoded_df['Outcome']


C:\Users\apath\anaconda3\lib\site-packages\pandas\core\common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

# Print size of train and test sets to check
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)


(7660, 41) (7660,)
(1916, 41) (1916,)


In [55]:
from sklearn import tree

model = tree.DecisionTreeClassifier()
model = model.fit(X_train, Y_train)

C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [56]:
Y_pred = model.predict(X_test)

print(Y_pred)

['Loss' 'Loss' 'Loss' ... 'Loss' 'Loss' 'Loss']


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [57]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(Y_test, Y_pred)
print('Accuracy on test set:', acc)

Accuracy on test set: 0.5245302713987474


In [58]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=1.0)
model.fit(X_train, Y_train)

C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


LogisticRegression()

In [59]:
from sklearn.metrics import accuracy_score

Y_pred = model.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print('Accuracy on test set:', acc)

Accuracy on test set: 0.494258872651357


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
from sklearn import svm

model = svm.SVC(kernel='linear', C=1.0)

model.fit(X_train, Y_train)

C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score

Y_pred = model.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print('Accuracy on test set:', acc)

In [ ]:
#approach2
X=one_hot_encoded_df[['Distance',
 'Exit Velocity',
 'Launch Angle',
 ('sum', 'AB'),
 ('sum', 'BB'),
 ('sum', 'H'),
 ('sum', 'PA'),
 ('sum', 'TB'),
 'HBreak',
 'MPH',
 'Spin',
 'VBreak',
 ('sum', 'Ball'),
 ('sum', 'Hit By Pitch'),
 ('sum', 'In play, no out'),
 ('sum', 'In play, out(s)'),
 ('sum', 'In play, run(s)'),
 ('sum', 'Strike'),'Team_AL',
 'Team_Angels',
 'Team_Astros',
 'Team_Athletics',
 'Team_Blue Jays',
 'Team_Braves',
 'Team_Brewers',
 'Team_Cardinals',
 'Team_Cubs',
 'Team_D-Backs',
 'Team_Dodgers',
 'Team_Giants',
 'Team_Guardians',
 'Team_Mariners',
 'Team_Marlins',
 'Team_Mets',
 'Team_NL',
 'Team_Nationals',
 'Team_Orioles',
 'Team_Padres',
 'Team_Phillies',
 'Team_Pirates',
 'Team_Rangers',
 'Team_Rays',
 'Team_Red Sox',
 'Team_Reds',
 'Team_Rockies',
 'Team_Royals',
 'Team_Tigers',
 'Team_Twins',
 'Team_White Sox',
 'Team_Yankees']]
Y=one_hot_encoded_df['Outcome']


In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10)

# Print size of train and test sets to check
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)


(1794, 50) (1794,)
(770, 50) (770,)


In [70]:
from sklearn import tree

model = tree.DecisionTreeClassifier()
model = model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score

acc = accuracy_score(Y_test, Y_pred)
print('Accuracy on test set:', acc)

Accuracy on test set: 0.5233766233766234


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [97]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver = 'liblinear',C=0.01, random_state=0)
model.fit(X_train, Y_train)

from sklearn.metrics import accuracy_score

Y_pred = model.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print('Accuracy on test set:', acc)

Accuracy on test set: 0.6051948051948052


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


In [100]:
from sklearn import svm

C = 0.1
model = svm.SVC(kernel='linear', C=C)

model.fit(X_train, Y_train)

from sklearn.metrics import accuracy_score

Y_pred = model.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print('Accuracy on test set:', acc)

C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(


Accuracy on test set: 0.6077922077922078


C:\Users\apath\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  warnings.warn(
